# Library

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# fold path

In [6]:
drive = 'D'
raw_fold = f'{drive}:/data/big2/BD1/raw'
ods_fold = f'{drive}:/data/big2/ODS'
sample_fold = f'{drive}:/data/big2/BD1/sample'
analysis_fold = f'{drive}:/data/big2/BD1/an'

# Load

## 병합 result 최종 파일

In [12]:
# about 16.5s
name = '4등급 차량 리스트 양식(전체) result 최종 Grade 추가(2023.04.13_AM.08.39.05)'
file_name = f'{name}.csv'
df = pd.read_csv(os.path.join(raw_fold, file_name), low_memory=False, encoding='cp949')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502235 entries, 0 to 1502234
Data columns (total 33 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   차대번호      1502235 non-null  object 
 1   차량번호      1502235 non-null  object 
 2   제원관리번호    1502235 non-null  object 
 3   차종        1502235 non-null  object 
 4   용도        1502235 non-null  object 
 5   최초등록일     1502235 non-null  int64  
 6   연식        1502235 non-null  int64  
 7   제작일자      1335178 non-null  float64
 8   검사유효일     1502223 non-null  float64
 9   배출가스인증번호  1382222 non-null  object 
 10  등급        1502235 non-null  int64  
 11  차명        1502235 non-null  object 
 12  차종분류      1502235 non-null  object 
 13  차종유형      1502235 non-null  object 
 14  자동차형식     1501958 non-null  object 
 15  제작사명      1492590 non-null  object 
 16  연료        1502235 non-null  object 
 17  엔진형식      1500400 non-null  object 
 18  총중량       1502234 non-null  float64
 19  적재중량      1501728 non

# 전처리

In [13]:
df.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', 'fuel', 'Grade'],
      dtype='object')

In [14]:
df['연료'].value_counts(dropna=False)

경유             1153813
휘발유             322983
LPG(액화석유가스)      25400
기타연료                37
알코올                  1
CNG(압축천연가스)          1
Name: 연료, dtype: int64

In [15]:
df['등급_수정'].value_counts(dropna=False)

4.00    1149290
NaN      348422
3.00       4396
5.00        127
Name: 등급_수정, dtype: int64

## 중복 차대번호 확인
- 최종등록일자, 배출가스검사유효일자 최신 데이터 사용

In [16]:
df.shape[0], len(df['차대번호'].unique())

(1502235, 1502213)

## 차종 확인

In [17]:
df['차종'].value_counts(dropna=False)

승용    854583
화물    496664
승합    142835
특수      8153
Name: 차종, dtype: int64

## 차종유형 확인

In [18]:
df['차종유형'].value_counts()

중형    706291
소형    555000
대형    213697
경형     27247
Name: 차종유형, dtype: int64

## 연식 확인

In [19]:
df['연식'].describe()

count   1502235.00
mean       2005.89
std           6.83
min        1901.00
25%        2006.00
50%        2009.00
75%        2010.00
max        2020.00
Name: 연식, dtype: float64

## 연료별 차종별 차종유형 확인

In [81]:
df.groupby(['연료', '차종', '차종유형'])['차대번호'].count()

연료           차종  차종유형
CNG(압축천연가스)  화물  대형           1
LPG(액화석유가스)  승용  경형          51
                 대형         845
                 소형         131
                 중형       11221
             승합  경형        2148
                 소형         386
                 중형        4501
             특수  중형           2
             화물  경형        3511
                 대형           1
                 소형        1792
                 중형         811
경유           승용  경형           1
                 대형      147969
                 소형       14589
                 중형      364672
             승합  대형        3470
                 중형      129074
             특수  대형        5578
                 소형        2045
                 중형         524
             화물  대형       21268
                 소형      426704
                 중형       37919
기타연료         승용  대형           3
                 소형          13
                 중형          20
             화물  소형           1
알코올          승용  대형           1
휘발유          승용  경

In [85]:
for fuel in ['경유', '휘발유', 'LPG(액화석유가스)']:
    temp1 = df[df['연료'] == fuel].reset_index(drop=True)
    for car_type in temp1['차종'].unique():
        temp2 = temp1[temp1['차종'] == car_type].reset_index(drop=True)
        car_size = list(temp2['차종유형'].unique())
        print(f'{fuel} - {car_type}의 차종유형 : {car_size}')

경유 - 승용의 차종유형 : ['대형', '중형', '소형', '경형']
경유 - 특수의 차종유형 : ['대형', '소형', '중형']
경유 - 승합의 차종유형 : ['대형', '중형']
경유 - 화물의 차종유형 : ['대형', '소형', '중형']
휘발유 - 승용의 차종유형 : ['소형', '대형', '중형', '경형']
휘발유 - 화물의 차종유형 : ['경형', '소형', '대형', '중형']
휘발유 - 승합의 차종유형 : ['경형', '소형', '중형', '대형']
휘발유 - 특수의 차종유형 : ['중형']
LPG(액화석유가스) - 승합의 차종유형 : ['중형', '경형', '소형']
LPG(액화석유가스) - 승용의 차종유형 : ['중형', '대형', '소형', '경형']
LPG(액화석유가스) - 화물의 차종유형 : ['소형', '경형', '중형', '대형']
LPG(액화석유가스) - 특수의 차종유형 : ['중형']


### 차령 정보 계산 및 데이터 추가

In [20]:
current_yr = int(datetime.today().strftime("%Y"))
current_yr

2023

In [21]:
df['차령'] = current_yr - df['연식']
df['차령'].describe()

count   1502235.00
mean         17.11
std           6.83
min           3.00
25%          13.00
50%          14.00
75%          17.00
max         122.00
Name: 차령, dtype: float64

## 시도명, 시군구명 수정
- 시군구명 앞지역명만 추출
- 세종특별자치시 -> 세종시
- 여주군 -> 여주시
- 청원군 -> 청원시
- 인천광역시 남구 -> 미추홀구

In [22]:
df[['시도명', '시군구명', '차종']].isnull().sum()

시도명     0
시군구명    0
차종      0
dtype: int64

In [23]:
df['시도명'].unique()

array(['대전광역시', '부산광역시', '경기도', '제주특별자치도', '경상남도', '강원도', '전라남도', '인천광역시',
       '서울특별시', '충청북도', '충청남도', '대구광역시', '전라북도', '울산광역시', '경상북도', '광주광역시',
       '세종특별자치시'], dtype=object)

In [27]:
df[['시도명', '시군구명']].iloc[10:20]

,시도명,시군구명
10,경기도,남양주시
11,경기도,화성시
12,경기도,안성시
13,경기도,평택시
14,경기도,안산시 단원구
15,부산광역시,부산진구
16,인천광역시,강화군
17,서울특별시,서초구
18,부산광역시,금정구
19,경기도,성남시 분당구


In [28]:
df['시군구명_수정'] = df['시군구명'].str.split(' ').str[0]

In [30]:
df[['시도명', '시군구명', '시군구명_수정']].iloc[10:20]

,시도명,시군구명,시군구명_수정
10,경기도,남양주시,남양주시
11,경기도,화성시,화성시
12,경기도,안성시,안성시
13,경기도,평택시,평택시
14,경기도,안산시 단원구,안산시
15,부산광역시,부산진구,부산진구
16,인천광역시,강화군,강화군
17,서울특별시,서초구,서초구
18,부산광역시,금정구,금정구
19,경기도,성남시 분당구,성남시


In [31]:
df.loc[df['시군구명_수정'] == '세종특별자치시', '시군구명_수정'].shape

(8122,)

In [32]:
df.loc[df['시군구명_수정'] == '세종특별자치시', '시군구명_수정'] = '세종시'

In [33]:
df.loc[df['시군구명_수정'] == '세종특별자치시', '시군구명_수정'].shape

(0,)

In [44]:
df.loc[df['시군구명_수정'] == '여주군', '시군구명_수정'].shape

(0,)

In [45]:
df.loc[df['시군구명_수정'] == '여주군', '시군구명_수정'] = '여주시'

In [46]:
df.loc[df['시군구명_수정'] == '여주군', '시군구명_수정'].shape

(0,)

In [47]:
df.loc[df['시군구명_수정'] == '청원군', '시군구명_수정'].shape

(4,)

In [48]:
df.loc[df['시군구명_수정'] == '청원군', '시군구명_수정'] = '청주시'

In [49]:
df.loc[df['시군구명_수정'] == '청원군', '시군구명_수정'].shape

(0,)

In [63]:
df.loc[(df['시도명'] == '인천광역시') & (df['시군구명_수정'] == '남구'), '시군구명_수정'].shape

(9,)

In [64]:
df.loc[(df['시도명'] == '인천광역시') & (df['시군구명_수정'] == '남구'), '시군구명_수정'] = '미추홀구'

In [65]:
df.loc[(df['시도명'] == '인천광역시') & (df['시군구명_수정'] == '남구'), '시군구명_수정'].shape

(0,)

# 자동차-엔진 가열(Hot-start) 배출

## 배출량 산정
$E_{i, j} = VKT \times {EF_i \over {1000}} \times DF \times (1 - {R \over {100}})$

$E_{i, j}$ : 자동차 $j$의 도로주행시 발생한 오염물질 $i$의 배출량($kg/yr$)

$VKT$ : 주행거리($km/yr$)

$EF_i$ : 차종별, 연료별, 연식별, 차속별 배출계수($g/km$)

$DF$ : 열화계수

$R$ : 저감장치 부착 효율(%)

### 주행거리(VKT)
- "용도별_차종별_시군구별_자동차주행거리" 자료(KOSIS)의 1일 평균주행거리 * 365 = 주행거리(km/yr)

#### [로드] 1일평균주행거리(2021)

In [35]:
# 1일 평균 주행거리 자료 로드
run = pd.read_excel(os.path.join(raw_fold, '용도별_차종별_시군구별_자동차1일평균주행거리(2021).xlsx'))
run.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1230 entries, 0 to 1229
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   시도명     1230 non-null   object 
 1   시군구명    1230 non-null   object 
 2   차종      1230 non-null   object 
 3   전체      1230 non-null   float64
dtypes: float64(1), object(3)
memory usage: 38.6+ KB


In [36]:
run.tail()

,시도명,시군구명,차종,전체
1225,제주특별자치도,서귀포시,합계,35.10
1226,제주특별자치도,서귀포시,승용,36.10
1227,제주특별자치도,서귀포시,승합,37.20
1228,제주특별자치도,서귀포시,화물,32.50
1229,제주특별자치도,서귀포시,특수,38.40


In [66]:
len(set(df['시군구명_수정'].unique()) - set(run['시군구명'].unique()))

0

In [67]:
df.shape

(1502235, 35)

In [68]:
df1 = df.merge(run, left_on=['시도명', '시군구명_수정', '차종'], right_on=['시도명', '시군구명', '차종'], how='left')
df1.shape

(1502235, 37)

In [69]:
df1.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명_x', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', 'fuel', 'Grade', '차령',
       '시군구명_수정', '시군구명_y', '전체'],
      dtype='object')

In [70]:
df1.drop('시군구명_y', axis=1, inplace=True)
df1.rename(columns={'시군구명_x':'시군구명', '전체':'1일평균주행거리'}, inplace=True)
df1.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', 'fuel', 'Grade', '차령',
       '시군구명_수정', '1일평균주행거리'],
      dtype='object')

In [71]:
df1['1일평균주행거리'].isnull().sum()

0

In [72]:
df1[['시도명', '시군구명', '시군구명_수정', '차종', '1일평균주행거리']].tail()

,시도명,시군구명,시군구명_수정,차종,1일평균주행거리
1502230,대구광역시,달성군,달성군,승용,37.90
1502231,대구광역시,북구,북구,승용,36.80
1502232,대구광역시,동구,동구,승용,37.40
1502233,대구광역시,서구,서구,승용,36.50
1502234,대구광역시,달서구,달서구,승용,36.20


In [73]:
df1['VKT'] = df1['1일평균주행거리'] * 365

In [74]:
df1.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', 'fuel', 'Grade', '차령',
       '시군구명_수정', '1일평균주행거리', 'VKT'],
      dtype='object')

In [75]:
df1[['시도명', '시군구명', '차종', 'VKT']].tail()

,시도명,시군구명,차종,VKT
1502230,대구광역시,달성군,승용,13833.50
1502231,대구광역시,북구,승용,13432.00
1502232,대구광역시,동구,승용,13651.00
1502233,대구광역시,서구,승용,13322.50
1502234,대구광역시,달서구,승용,13213.00


In [76]:
df1['VKT'].isnull().sum()

0

### 배출가스 열화계수(DF) 계산
- 필요한 정보 : 연료, 차종, 차종유형, 연식, 차령

In [80]:
df1['연료'].unique()

array(['경유', '휘발유', 'LPG(액화석유가스)', '기타연료', '알코올', 'CNG(압축천연가스)'],
      dtype=object)

In [77]:
DF_col = ['연료', '차종', '차종유형', '연식', '차령']

In [78]:
df1[DF_col].head()

,연료,차종,차종유형,연식,차령
0,경유,승용,대형,2006,17
1,경유,승용,대형,2006,17
2,경유,승용,대형,2006,17
3,경유,승용,대형,2006,17
4,경유,승용,대형,2006,17


In [79]:
df1[DF_col].values[0]

array(['경유', '승용', '대형', 2006, 17], dtype=object)

#### 열화계수 코드

In [42]:
# about 3.5s
DF_CO_list = []
DF_HC_list = []
DF_PM_list = []
DF_NOx_list = []
# '연료', '차종', '차종유형', '연식', '차령'
for fuel, car_type, car_size, car_birth, car_age in tqdm(df1[DF_col].values):    
    if fuel == '경유':
        if car_type == '승용':
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
            DF_CO_list.append(DF_CO)
            DF_HC_list.append(DF_HC)
            DF_PM_list.append(DF_PM)
            DF_NOx_list.append(DF_NOx)
        elif car_type == '승합':
            if car_size == '소형' or car_size == '중형':
                warranty = 5
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.05
                    DF_HC = 1 + (car_age - warranty)*0.05
                    DF_PM = 1 + (car_age - warranty)*0.05
                    DF_NOx = 1 + (car_age - warranty)*0.02
                    if DF_CO > 1.5:
                        DF_CO = 1.5
                    if DF_HC > 1.5:
                        DF_HC = 1.5
                    if DF_PM > 1.5:
                        DF_PM = 1.5
                    if DF_NOx > 1.2:
                        DF_NOx = 1.2
                DF_CO_list.append(DF_CO)
                DF_HC_list.append(DF_HC)
                DF_PM_list.append(DF_PM)
                DF_NOx_list.append(DF_NOx)
            elif car_size == '대형':
                warranty = 3
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.05
                    DF_HC = 1 + (car_age - warranty)*0.05
                    DF_PM = 1 + (car_age - warranty)*0.05
                    DF_NOx = 1 + (car_age - warranty)*0.02
                    if DF_CO > 1.5:
                        DF_CO = 1.5
                    if DF_HC > 1.5:
                        DF_HC = 1.5
                    if DF_PM > 1.5:
                        DF_PM = 1.5
                    if DF_NOx > 1.2:
                        DF_NOx = 1.2
                DF_CO_list.append(DF_CO)
                DF_HC_list.append(DF_HC)
                DF_PM_list.append(DF_PM)
                DF_NOx_list.append(DF_NOx)
        elif car_type == '화물':
            # car_size : 경, 소, 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
            DF_CO_list.append(DF_CO)
            DF_HC_list.append(DF_HC)
            DF_PM_list.append(DF_PM)
            DF_NOx_list.append(DF_NOx)
        elif car_type == '특수':
            # car_size : 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
            DF_CO_list.append(DF_CO)
            DF_HC_list.append(DF_HC)
            DF_PM_list.append(DF_PM)
            DF_NOx_list.append(DF_NOx)
    elif fuel == '휘발유' | fuel == 'LPG(액화석유가스)':
        if car_type == '승용':
            # 경, 소, 중, 대
            if car_birth <= 2000:
                warranty = 5
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.1
                    DF_HC = 1 + (car_age - warranty)*0.1
                    DF_PM = 1 + (car_age - warranty)*0.1
                    DF_NOx = 1 + (car_age - warranty)*0.1
                    if DF_CO > 2.0:
                        DF_CO = 2.0
                    if DF_HC > 2.0:
                        DF_HC = 2.0
                    if DF_PM > 2.0:
                        DF_PM = 2.0
                    if DF_NOx > 2.0:
                        DF_NOx = 2.0
                DF_CO_list.append(DF_CO)
                DF_HC_list.append(DF_HC)
                DF_PM_list.append(DF_PM)
                DF_NOx_list.append(DF_NOx)
            elif car_birth >= 2001:
                warranty = 10
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.1
                    DF_HC = 1 + (car_age - warranty)*0.1
                    DF_PM = 1 + (car_age - warranty)*0.1
                    DF_NOx = 1 + (car_age - warranty)*0.1
                    if DF_CO > 2.0:
                        DF_CO = 2.0
                    if DF_HC > 2.0:
                        DF_HC = 2.0
                    if DF_PM > 2.0:
                        DF_PM = 2.0
                    if DF_NOx > 2.0:
                        DF_NOx = 2.0
                DF_CO_list.append(DF_CO)
                DF_HC_list.append(DF_HC)
                DF_PM_list.append(DF_PM)
                DF_NOx_list.append(DF_NOx)
        elif car_type == '승합':
            if car_size == '경형':
                warranty = 5
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.1
                    DF_HC = 1 + (car_age - warranty)*0.1
                    DF_PM = 1 + (car_age - warranty)*0.1
                    DF_NOx = 1 + (car_age - warranty)*0.1
                    if DF_CO > 2.0:
                        DF_CO = 2.0
                    if DF_HC > 2.0:
                        DF_HC = 2.0
                    if DF_PM > 2.0:
                        DF_PM = 2.0
                    if DF_NOx > 2.0:
                        DF_NOx = 2.0
                DF_CO_list.append(DF_CO)
                DF_HC_list.append(DF_HC)
                DF_PM_list.append(DF_PM)
                DF_NOx_list.append(DF_NOx)
            elif car_size == '소형' | car_size == '중형':
                if car_birth <= 2005:
                    warranty = 5
                    if car_age <= warranty:
                        DF_CO = 1
                        DF_HC = 1
                        DF_PM = 1
                        DF_NOx = 1
                    else:
                        DF_CO = 1 + (car_age - warranty)*0.1
                        DF_HC = 1 + (car_age - warranty)*0.1
                        DF_PM = 1 + (car_age - warranty)*0.1
                        DF_NOx = 1 + (car_age - warranty)*0.1
                        if DF_CO > 2.0:
                            DF_CO = 2.0
                        if DF_HC > 2.0:
                            DF_HC = 2.0
                        if DF_PM > 2.0:
                            DF_PM = 2.0
                        if DF_NOx > 2.0:
                            DF_NOx = 2.0
                    DF_CO_list.append(DF_CO)
                    DF_HC_list.append(DF_HC)
                    DF_PM_list.append(DF_PM)
                    DF_NOx_list.append(DF_NOx)
                elif car_birth >= 2006:
                    warranty = 10
                    if car_age <= warranty:
                        DF_CO = 1
                        DF_HC = 1
                        DF_PM = 1
                        DF_NOx = 1
                    else:
                        DF_CO = 1 + (car_age - warranty)*0.1
                        DF_HC = 1 + (car_age - warranty)*0.1
                        DF_PM = 1 + (car_age - warranty)*0.1
                        DF_NOx = 1 + (car_age - warranty)*0.1
                        if DF_CO > 2.0:
                            DF_CO = 2.0
                        if DF_HC > 2.0:
                            DF_HC = 2.0
                        if DF_PM > 2.0:
                            DF_PM = 2.0
                        if DF_NOx > 2.0:
                            DF_NOx = 2.0
                    DF_CO_list.append(DF_CO)
                    DF_HC_list.append(DF_HC)
                    DF_PM_list.append(DF_PM)
                    DF_NOx_list.append(DF_NOx)
        elif car_type == '화물':
            # 경, 소, 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.1
                DF_HC = 1 + (car_age - warranty)*0.1
                DF_PM = 1 + (car_age - warranty)*0.1
                DF_NOx = 1 + (car_age - warranty)*0.1
                if DF_CO > 2.0:
                    DF_CO = 2.0
                if DF_HC > 2.0:
                    DF_HC = 2.0
                if DF_PM > 2.0:
                    DF_PM = 2.0
                if DF_NOx > 2.0:
                    DF_NOx = 2.0
            DF_CO_list.append(DF_CO)
            DF_HC_list.append(DF_HC)
            DF_PM_list.append(DF_PM)
            DF_NOx_list.append(DF_NOx)
        elif car_type == '특수':
            # 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.1
                DF_HC = 1 + (car_age - warranty)*0.1
                DF_PM = 1 + (car_age - warranty)*0.1
                DF_NOx = 1 + (car_age - warranty)*0.1
                if DF_CO > 2.0:
                    DF_CO = 2.0
                if DF_HC > 2.0:
                    DF_HC = 2.0
                if DF_PM > 2.0:
                    DF_PM = 2.0
                if DF_NOx > 2.0:
                    DF_NOx = 2.0
            DF_CO_list.append(DF_CO)
            DF_HC_list.append(DF_HC)
            DF_PM_list.append(DF_PM)
            DF_NOx_list.append(DF_NOx)

100%|██████████| 1149290/1149290 [00:03<00:00, 328462.93it/s]


In [43]:
df1.shape[0]

1149290

In [44]:
len(DF_CO_list), len(DF_HC_list), len(DF_PM_list), len(DF_NOx_list)

(1149290, 1149290, 1149290, 1149290)

In [45]:
df1['DF_CO'] = DF_CO_list
df1['DF_HC'] = DF_HC_list
df1['DF_PM'] = DF_PM_list
df1['DF_NOx'] = DF_NOx_list

In [46]:
check_DF_col = ['DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx']
df1.loc[df1['차령'] < 10, DF_col + check_DF_col].head()

,연료,차종,차종유형,연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx
1143229,경유,화물,중형,2018,5,1.00,1.00,1.00,1.00
1145884,경유,화물,대형,2018,5,1.00,1.00,1.00,1.00


### 저감장치 부착 효율(R) 계산
- DPF유무_수정 : 유 -> DPF로 가정하고 진행
- 휘발성 유기 화합물(Volatile Organic Compounds:VOC) : 생활주변에서 흔히 사용하는 탄화수소류가 거의 해당됨.
- 저감장치별 물질 제거 효율(%)

    장치종류|CO|VOC|PM
    :-:|:-:|:-:|:-:
    DPF|99.5|90|83.6
    pDPF|94.6|89.3|56
    DOC|85.4|72|35

In [1]:
df1['DPF유무_수정'].value_counts(dropna=False)

NameError: name 'df1' is not defined

In [48]:
df1['DPF유무_수정'].values[0]

'무'

In [49]:
R_CO_list = []
R_HC_list = []
R_PM_list = []
for dpf in tqdm(df1['DPF유무_수정'].values):
    if dpf == '유':
        R_CO = 99.5
        R_HC = 90
        R_PM = 83.6
    else:
        R_CO = 0
        R_HC = 0
        R_PM = 0
    R_CO_list.append(R_CO)
    R_HC_list.append(R_HC)
    R_PM_list.append(R_PM)

100%|██████████| 1149290/1149290 [00:00<00:00, 1322688.74it/s]


In [50]:
df1.shape[0]

1149290

In [51]:
len(R_CO_list), len(R_HC_list), len(R_PM_list)

(1149290, 1149290, 1149290)

In [52]:
df1['R_CO'] = R_CO_list
df1['R_HC'] = R_HC_list
df1['R_PM'] = R_PM_list

In [53]:
check_R_col = ['R_CO', 'R_HC', 'R_PM']
df1.loc[df1['DPF유무_수정'] == '유', DF_col + check_DF_col + check_R_col].head()

,연료,차종,차종유형,연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx,R_CO,R_HC,R_PM
3207,경유,승용,대형,2006,17,1.50,1.50,1.50,1.20,99.50,90,83.60
3208,경유,승용,대형,2006,17,1.50,1.50,1.50,1.20,99.50,90,83.60
3209,경유,승용,대형,2007,16,1.50,1.50,1.50,1.20,99.50,90,83.60
3210,경유,승용,대형,2007,16,1.50,1.50,1.50,1.20,99.50,90,83.60
3211,경유,승용,대형,2006,17,1.50,1.50,1.50,1.20,99.50,90,83.60


### 배출계수(EFi)


In [4]:
EFi_col = ['차종', '차종분류', '연료', '연식']
# KOSIS 차량속도(2017) 참고
V = 60.9
# 국가 대기오염물질 배출량 산정방법 편람(V)(2022) 부록 참고(for PM-2.5)
k = 0.92

In [ ]:
EFi_CO_list = []
EFi_HC_list = []
EFi_NOx_list = []
EFi_PM10_list = []
EFi_PM2_5_list = []
for car_type, car_size, fuel, car_birth in tqdm(df1[EFi_col].values):
    if fuel == '휘발유':
        if (car_type == '승용') & (car_size == '경형'):
            if car_birth <= 1996:
                if V <= 65:
                    EFi_CO = 59.783 * (V**-1.0007)
                else:
                    EFi_CO = 0.0874 * V - 3.5618
                EFi_HC = 7.6244 * (V**-0.8364)
                EFi_NOx = 2.6754 * (V**-0.3236)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1997 <= car_birth <= 1999:
                if V <= 65:
                    EFi_CO = 59.783 * (V**-1.0007)
                else:
                    EFi_CO = 0.0874 * V - 3.5618
                EFi_HC = 8.6275 * (V**-1.0722)
                EFi_NOx = 3.2294 * (V**-0.5763)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2000 <= car_birth <= 2002:
                if V <= 65:
                    EFi_CO = 60.556 * (V**-1.2501)
                else:
                    EFi_CO = -0.0006 * V + 0.5753
                EFi_HC = 5.1835 * (V**-1.1889)
                EFi_NOx = 1.7525 * (V**-0.6481)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                if V <= 65:
                    EFi_CO = 60.556 * (V**-1.2501)
                else:
                    EFi_CO = -0.0006 * V + 0.5753
                EFi_HC = 0.7446 * (V**-0.9392)
                EFi_NOx = 0.3403 * (V**-0.5455)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth <= 2008:
                if V <= 45:
                    EFi_CO = 4.9952 * (V**-0.8461)
                else:
                    EFi_CO = -0.0001 * V**2 + 0.0229 * V - 0.5701
                EFi_HC = 0.2958 * (V**-0.7830)
                EFi_NOx = 0.4819 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2009 <= car_birth <= 2011:
                if V <= 45:
                    EFi_CO = 4.5956 * (V**-0.8461)
                else:
                    EFi_CO = -9.2000*(10**-5) * (V**2) + 2.1068*(10**-2) * V - 5.2449*(10**-1)
                EFi_HC = 0.2662 * (V**-0.7830)
                EFi_NOx = 0.4476 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2012 <= car_birth <= 2013:
                if V <= 45:
                    EFi_CO = 4.4517 * (V**-0.8461)
                else:
                    EFi_CO = -8.9120*(10**-5) * (V**2) + 2.0408*(10**-2)*V - 5.0807*(10**-1)
                EFi_HC = 0.2556 * (V**-0.7830)
                EFi_NOx = 0.4353 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif car_birth == 2014:
                if V <= 45:
                    EFi_CO = 4.3079 * (V**-0.8461)
                else:
                    EFi_CO = -8.6240*(10**-5) * (V**2) + 1.9749*(10**-2)*V - 4.9165*(10**-1)
                EFi_HC = 0.2449 * (V**-0.7830)
                EFi_NOx = 0.4230 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2015 <= car_birth:
                if V <= 45:
                    EFi_CO = 4.164 * (V**-0.8461)
                else:
                    EFi_CO = -8.3360*(10**-5) * (V**2) + 1.9089*(10**-2)*V - 4.7524*(10**-1)
                EFi_HC = 0.2343 * (V**-0.7830)
                EFi_NOx = 0.4106 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif (car_type == '승용') & (car_size == '소형'):
            if car_birth <= 1986:
                EFi_CO = 247.00 * (V**-0.6651)
                EFi_HC = 15.953 * (V**-0.5059)
                EFi_NOx = 3.1140 * (V**-0.2278)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1987 <= car_birth <= 1990:
                EFi_CO = 36.169 * (V**-0.7587)
                EFi_HC = 15.607 * (V**-1.0423)
                EFi_NOx = 6.2007 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1991 <= car_birth <= 1999:
                EFi_CO = 111.67 * (V**-1.1566)
                EFi_HC = 32.017 * (V**-1.4171)
                EFi_NOx = 7.5244 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2000 <= car_birth <= 2002:
                EFi_CO = 22.356 * (V**-0.9068)
                EFi_HC = 0.8428 * (V**-0.8829)
                EFi_NOx = 1.2613 * (V**-0.3873)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                EFi_CO = 1.4898 * (V**-0.3837)
                EFi_HC = 0.1738 * (V**-0.7268)
                EFi_NOx = 0.1563 * (V**-0.2671)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth <= 2008:
                EFi_CO = 1.0000*(10**-4)*(V**2) - 7.1000*(10**-3)*V + 2.2450*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0633 * (V**-1.0484)
                else:
                    EFi_HC = 1.3200*(10**-6)*(V**2) - 1.8800*(10**-4)*V + 7.7000*(10**-3)
                EFi_NOx = -3.5000*(10**-6)*(V**2) + 3.3000*(10**-4)*V + 1.1200*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2009 <= car_birth <= 2011:
                EFi_CO = 9.2000*(10**-5)*(V**2) - 6.5320*(10**-3)*V + 2.0654*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0570 * (V**-1.0484)
                else:
                    EFi_HC = 1.1880*(10**-6)*(V**2) - 1.6920*(10**-4)*V + 6.9300*(10**-3)
                EFi_NOx = -3.2511*(10**-6)*(V**2) + 3.0653*(10**-4)*V + 1.0404*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2012 <= car_birth <= 2013:
                EFi_CO = 8.9120*(10**-5)*(V**2) - 6.3275*(10**-3)*V + 2.0007*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0547 * (V**-1.0484)
                else:
                    EFi_HC = 1.1405*(10**-6)*(V**2) - 1.6243*(10**-4)*V + 6.6528*(10**-3)
                EFi_NOx = -3.1615*(10**-6)*(V**2) + 2.9809*(10**-4)*V + 1.0117*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif car_birth == 2014:
                EFi_CO = 8.6240*(10**-5)*(V**2) - 6.1230*(10**-3)*V + 1.9361*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0524 * (V**-1.0484)
                else:
                    EFi_HC = 1.0930*(10**-6)*(V**2) - 1.5566*(10**-4)*V + 6.3756*(10**-3)
                EFi_NOx = -3.0719*(10**-6)*(V**2) + 2.8964*(10**-4)*V + 9.8301*(10**-3)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2015 <= car_birth:
                EFi_CO = 8.3360*(10**-5)*(V**2) - 5.9186*(10**-3)*V + 1.8714*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0501 * (V**-1.0484)
                else:
                    EFi_HC = 1.0500*(10**-6)*(V**2) - 1.4890*(10**-4)*V + 6.09840*(10**-3)
                EFi_NOx = -2.9823*(10**-6)*(V**2) + 2.8119*(10**-4)*V + 9.5434*(10**-3)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif (car_type == '승용') & (car_size == '중형'):
            if car_birth <= 1986:
                EFi_CO = 247.00 * (V**-0.6651)
                EFi_HC = 15.953 * (V**-0.5059)
                EFi_NOx = 3.1140 * (V**-0.2278)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1987 <= car_birth <= 1990:
                EFi_CO = 36.169 * (V**-0.7587)
                EFi_HC = 15.607 * (V**-1.0423)
                EFi_NOx = 6.2007 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1991 <= car_birth <= 1999:
                EFi_CO = 51.555 * (V**-0.9531)
                EFi_HC = 31.816 * (V**-1.4804)
                EFi_NOx = 7.5244 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2000 <= car_birth <= 2002:
                EFi_CO = 29.921 * (V**-0.8868)
                EFi_HC = 7.9374 * (V**-1.3041)
                EFi_NOx = 1.8525 * (V**-0.4192)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                EFi_CO = 2.4938 * (V**-0.6106)
                EFi_HC = 0.4262 * (V**-1.0122)
                EFi_NOx = 0.1818 * (V**-0.4316)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth <= 2008:
                EFi_CO = 2.2900*(10**-5)*(V**2) - 1.6300*(10**-3)*V + 5.8300*(10**-2)
                if V <= 65.4:
                    EFi_HC = 0.0633 * (V**-1.0484)
                else:
                    EFi_HC = 1.3200*(10**-6)*(V**2) - 1.8800*(10**-4)*V + 7.7000*(10**-3)
                EFi_NOx = -3.5000*(10**-6)*(V**2) + 3.3000*(10**-4)*V + 1.1200*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2009 <= car_birth <= 2011:
                EFi_CO = 2.1068*(10**-5)*(V**2) - 1.4996*(10**-3)*V + 5.3636*(10**-2)
                if V <= 65.4:
                    EFi_HC = 0.0570 * (V**-1.0484)
                else:
                    EFi_HC = 1.1880*(10**-6)*(V**2) - 1.6920*(10**-4)*V + 6.9300*(10**-3)
                EFi_NOx = -3.2511*(10**-6)*(V**2) + 3.0653*(10**-4)*V + 1.0404*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2012 <= car_birth <= 2013:
                EFi_CO = 2.0408*(10**-5)*(V**2) - 1.4527*(10**-3)*V + 5.1957*(10**-2)
                if V <= 65.4:
                    EFi_HC = 0.0547 * (V**-1.0484)
                else:
                    EFi_HC = 1.1405*(10**-6)*(V**2) - 1.6243*(10**-4)*V + 6.6528*(10**-3)
                EFi_NOx = -3.1615*(10**-6)*(V**2) + 2.9809*(10**-4)*V + 1.0117*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif car_birth == 2014:
                EFi_CO = 1.9749*(10**-5)*(V**2) - 1.4057*(10**-3)*V + 5.0278*(10**-2)
                if V <= 65.4:
                    EFi_HC = 0.0524 * (V**-1.0484)
                else:
                    EFi_HC = 1.0930*(10**-6)*(V**2) - 1.5566*(10**-4)*V + 6.3756*(10**-3)
                EFi_NOx = -3.0719*(10**-6)*(V**2) + 2.8964*(10**-4)*V + 9.8301*(10**-3)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2015 <= car_birth:
                EFi_CO = 1.9089*(10**-5)*(V**2) - 1.3588*(10**-3)*V + 4.8599*(10**-2)
                if V <= 65.4:
                    EFi_HC = 0.0501 * (V**-1.0484)
                else:
                    EFi_HC = 1.0500*(10**-6)*(V**2) - 1.4890*(10**-4)*V + 6.0984*(10**-3)
                EFi_NOx = -2.9823*(10**-6)*(V**2) + 2.8119*(10**-4)*V + 9.5434*(10**-3)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif (car_type == '승용') & (car_size == '대형'):
            if car_birth <= 1986:
                EFi_CO = 247.00 * (V**-0.6651)
                EFi_HC = 15.953 * (V**-0.5059)
                EFi_NOx = 3.1140 * (V**-0.2278)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1987 <= car_birth <= 1990:
                EFi_CO = 36.169 * (V**-0.7587)
                EFi_HC = 15.607 * (V**-1.0423)
                EFi_NOx = 6.2007 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1991 <= car_birth <= 1999:
                EFi_CO = 51.555 * (V**-0.9531)
                EFi_HC = 31.816 * (V**-1.4804)
                EFi_NOx = 7.5244 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2000 <= car_birth <= 2002:
                EFi_CO = 29.921 * (V**-0.8868)
                EFi_HC = 7.9374 * (V**-1.3041)
                EFi_NOx = 1.8525 * (V**-0.4192)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                EFi_CO = 2.4938 * (V**-0.6106)
                EFi_HC = 0.4262 * (V**-1.0122)
                EFi_NOx = 0.1818 * (V**-0.4316)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth <= 2008:
                if V <= 65.4:
                    EFi_CO = 1.4082 * (V**-0.7728)
                else:
                    EFi_CO = 8.0000*(10**-5)*(V**2) - 1.2700*(10**-2)*V + 5.7510*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0633 * (V**-1.0484)
                else:
                    EFi_HC = 1.3200*(10**-6)*(V**2) - 1.8800*(10**-4)*V + 7.7000*(10**-3)
                EFi_NOx = -3.5000*(10**-6)*(V**2) + 3.3000*(10**-4)*V + 1.1200*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2009 <= car_birth <= 2011:
                if V <= 65.4:
                    EFi_CO = 1.2955 * (V**-0.7728)
                else:
                    EFi_CO = 7.3600*(10**-5)*(V**2) - 1.1684*(10**-2)*V + 5.2909*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0570 * (V**-1.0484)
                else:
                    EFi_HC = 1.1880*(10**-6)*(V**2) - 1.6920*(10**-4)*V + 6.9300*(10**-3)
                EFi_NOx = -3.2511*(10**-6)*(V**2) + 3.0653*(10**-4)*V + 1.0404*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2012 <= car_birth <= 2013:
                if V <= 65.4:
                    EFi_CO = 1.2550 * (V**-0.7728)
                else:
                    EFi_CO = 7.1296*(10**-5)*(V**2) - 1.1318*(10**-2)*V + 5.1253*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0547 * (V**-1.0484)
                else:
                    EFi_HC = 1.1405*(10**-6)*(V**2) - 1.6243*(10**-4)*V + 6.6528*(10**-3)
                EFi_NOx = -3.1615*(10**-6)*(V**2) + 2.9809*(10**-4)*V + 1.0117*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif car_birth == 2014:
                if V <= 65.4:
                    EFi_CO = 1.2144 * (V**-0.7728)
                else:
                    EFi_CO = 6.8992*(10**-5)*(V**2) - 1.0952*(10**-2)*V + 4.9597*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0524 * (V**-1.0484)
                else:
                    EFi_HC = 1.0930*(10**-6)*(V**2) - 1.5566*(10**-4)*V + 6.3756*(10**-3)
                EFi_NOx = -3.0719*(10**-6)*(V**2) + 2.8964*(10**-4)*V + 9.8301*(10**-3)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2015 <= car_birth:
                if V <= 65.4:
                    EFi_CO = 1.1739 * (V**-0.7728)
                else:
                    EFi_CO = 6.6688*(10**-5)*(V**2) - 1.0587*(10**-2)*V + 4.7940*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0501 * (V**-1.0484)
                else:
                    EFi_HC = 1.0500*(10**-6)*(V**2) - 1.4890*(10**-4)*V + 6.0984*(10**-3)
                EFi_NOx = -2.9823*(10**-6)*(V**2) + 2.8119*(10**-4)*V + 9.5434*(10**-3)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif ( (car_type == '승합') & (car_size == '경형') ) | ( (car_type == '화물') & (car_size == '경형') ):
            if car_birth <= 1996:
                if V <= 45:
                    EFi_CO = 11.249 * (V**-0.6579)
                else:
                    EFi_CO = 0.0003 * (V**2) + 0.0002 * V + 0.4136
                EFi_HC = 7.6244 * (V**-0.8364)
                EFi_NOx = 2.6754 * (V**-0.3236)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1997 <= car_birth <= 1999:
                if V <= 45:
                    EFi_CO = 16.965 * (V**-0.8461)
                else:
                    EFi_CO = -0.0003 * (V**2) + 0.0777 * V - 1.9363
                EFi_HC = 3.0285 * (V**-0.7830)
                EFi_NOx = 1.9923 * (V**-0.3889)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2000 <= car_birth <= 2002:
                if V <= 45:
                    EFi_CO = 9.9433 * (V**-0.8461)
                else:
                    EFi_CO = -0.0002 * (V**2) + 0.0455 * V - 1.1349
                EFi_HC = 1.8928 * (V**-0.7830)
                EFi_NOx = 1.2352 * (V**-0.3889)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                if V <= 45:
                    EFi_CO = 9.9433 * (V**-0.8461)
                else:
                    EFi_CO = -0.0002 * (V**2) + 0.0455 * V - 1.1349
                EFi_HC = 0.9227 * (V**-0.7830)
                EFi_NOx = 3.8859 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth:
                if V <= 45:
                    EFi_CO = 4.4952 * (V**-0.8461)
                else:
                    EFi_CO = -0.0001 * (V**2) + 0.0229 * V - 0.5701
                EFi_HC = 0.2958 * (V**-0.7830)
                EFi_NOx = 0.4819 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif (car_type == '승합') & (car_size == '소형'):
            if car_birth <= 1990:
                EFi_CO = 36.169 * (V**-0.7587)
                EFi_HC = 15.607 * (V**-1.0423)
                EFi_NOx = 6.2007 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1991 <= car_birth <= 1996:
                EFi_CO = 39.402 * (V**-0.8879)
                EFi_HC = 23.400 * (V**-1.4041)
                EFi_NOx = 7.5244 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1997 <= car_birth <= 1999:
                EFi_CO = 77.088 * (V**-1.2078)
                EFi_HC = 18.731 * (V**-1.5356)
                EFi_NOx = 4.4260 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif car_birth == 2000:
                EFi_CO = 41.669 * (V**-1.2078)
                EFi_HC = 14.190 * (V**-1.5356)
                EFi_NOx = 3.4578 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2001 <= car_birth <= 2002:
                EFi_CO = 41.669 * (V**-1.2078)
                EFi_HC = 11.920 * (V**-1.5356)
                EFi_NOx = 3.0649 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                EFi_CO = 37.573 * (V**-1.2078)
                EFi_HC = 3.1786 * (V**-1.5356)
                EFi_NOx = 1.4931 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth:
                EFi_CO = 32.899 * (V**-1.2078)
                EFi_HC = 2.7387 * (V**-1.5356)
                EFi_NOx = 1.1808 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif ( (car_type == '승합') & ((car_size == '중형') | (car_size == '대형')) ) | ( (car_type == '화물') & (car_size == '소형') ):
            if car_birth <= 1990:
                EFi_CO = 36.169 * (V**-0.7587)
                EFi_HC = 15.607 * (V**-1.0423)
                EFi_NOx = 6.2007 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1991 <= car_birth <= 1996:
                EFi_CO = 39.402 * (V**-0.8879)
                EFi_HC = 23.400 * (V**-1.4041)
                EFi_NOx = 7.5244 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1997 <= car_birth <= 1999:
                EFi_CO = 77.088 * (V**-1.2078)
                EFi_HC = 18.731 * (V**-1.5356)
                EFi_NOx = 4.4260 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2000 <= car_birth <= 2002:
                EFi_CO = 41.669 * (V**-1.2078)
                EFi_HC = 14.190 * (V**-1.5356)
                EFi_NOx = 3.4578 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                EFi_CO = 36.578 * (V**-1.2078)
                EFi_HC = 3.0337 * (V**-1.5356)
                EFi_NOx = 2.0104 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth:
                EFi_CO = 14.202 * (V**-1.2078)
                EFi_HC = 1.2233 * (V**-1.5356)
                EFi_NOx = 0.2493 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif (car_type == '화물') & ( (car_size == '중형') | (car_size == '대형')):
            if 1993 <= car_birth:
                if V <= 65:
                    EFi_CO = 70
                    EFi_HC = 7
                    EFi_NOx = 4.5
                else:
                    EFi_CO = 55
                    EFi_HC = 3.5
                    EFi_NOx = 7.5
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
    elif fuel == '경유':
        if (car_type == '승용') & (car_size == '경형'):
            if car_birth <= 2005:
                EFi_CO = 0.7392 * (V**-0.7524)
                EFi_HC = 0.0989 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0839 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0839 * (V**-0.3420)
            elif 2006 <= car_birth <= 2010:
                EFi_CO = 0.5775 * (V**-0.7524)
                EFi_HC = 0.0825 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0420 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0420 * (V**-0.3420)
            elif 2011 <= car_birth <= 2015:
                EFi_CO = 0.5141 * (V**-0.6792)
                EFi_HC = 0.3713 * (V**-0.7513)
                EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
            elif 2016 <= car_birth:
                EFi_CO = 0.4574 * (V**-0.5215)
                EFi_HC = 0.1300 * (V**-0.7265)
                EFi_NOx = 2.7702 * (V**-0.3869)
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
        elif (car_type == '승용') & (car_size == '소형'):
            if car_birth <= 2004:
                EFi_CO = 5.9672 * (V**-0.9534)
                EFi_HC = 0.6523 * (V**-1.0167)
                EFi_NOx = 24.3491 * (V**-0.7277)
                if V <= 65:
                    EFi_PM10 = 0.3861 * (V**-0.5093)
                    EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                else:
                    EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
            elif car_birth == 2005:
                EFi_CO = 0.7392 * (V**-0.7524)
                EFi_HC = 0.0989 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0839 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0839 * (V**-0.3420)
            elif 2006 <= car_birth <= 2010:
                EFi_CO = 0.5775 * (V**-0.7524)
                EFi_HC = 0.0825 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0420 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0420 * (V**-0.3420)
            elif 2011 <= car_birth <= 2015:
                EFi_CO = 0.5141 * (V**-0.6792)
                EFi_HC = 0.3713 * (V**-0.7513)
                EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
            elif 2016 <= car_birth:
                EFi_CO = 0.4574 * (V**-0.5215)
                EFi_HC = 0.1300 * (V**-0.7265)
                EFi_NOx = 2.7702 * (V**-0.3869)
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
        elif (car_type == '승용') & (car_size == '중형'):
            if car_birth <= 2004:
                EFi_CO = 5.9672 * (V**-0.9534)
                EFi_HC = 0.6523 * (V**-1.0167)
                EFi_NOx = 24.3491 * (V**-0.7277)
                if V <= 65:
                    EFi_PM10 = 0.3861 * (V**-0.5093)
                    EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                else:
                    EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
            elif car_birth == 2005:
                EFi_CO = 0.6930 * (V**-0.7524)
                EFi_HC = 0.1865 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0723 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0723 * (V**-0.3420)
            elif 2006 <= car_birth <= 2010:
                EFi_CO = 0.5414 * (V**-0.7524)
                EFi_HC = 0.0927 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0396 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0396 * (V**-0.3420)
            elif 2011 <= car_birth <= 2015:
                EFi_CO = 0.5141 * (V**-0.6792)
                EFi_HC = 0.3713 * (V**-0.7513)
                EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
            elif 2016 <= car_birth:
                EFi_CO = 0.4574 * (V**-0.5215)
                EFi_HC = 0.1300 * (V**-0.7265)
                EFi_NOx = 2.7702 * (V**-0.3869)
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
        elif (car_type == '승용') & (car_size == '대형'):
            if car_birth <= 2004:
                EFi_CO = 5.9672 * (V**-0.9534)
                EFi_HC = 0.6523 * (V**-1.0167)
                EFi_NOx = 24.3491 * (V**-0.7277)
                if V <= 65:
                    EFi_PM10 = 0.3861 * (V**-0.5093)
                    EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                else:
                    EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
            elif car_birth == 2005:
                EFi_CO = 0.9609 * (V**-0.7524)
                EFi_HC = 0.1865 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0723 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0723 * (V**-0.3420)
            elif 2006 <= car_birth <= 2010:
                EFi_CO = 0.7507 * (V**-0.7524)
                EFi_HC = 0.1554 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0361 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0361 * (V**-0.3420)
            elif 2011 <= car_birth <= 2015:
                EFi_CO = 0.5141 * (V**-0.6792)
                EFi_HC = 0.3713 * (V**-0.7513)
                EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
            elif 2016 <= car_birth:
                EFi_CO = 0.4574 * (V**-0.5215)
                EFi_HC = 0.1300 * (V**-0.7265)
                EFi_NOx = 2.7702 * (V**-0.3869)
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
        elif (car_type == '승합') & (car_size == '소형'):
            if car_birth <= 1990:
                if V <= 65.4:
                    EFi_CO = 3.4539 * (V**-0.4266)
                else:
                    EFi_CO = 0.0051 * V + 0.2212
                EFi_HC = 0.9835 * (V**-0.5096)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 1.1412 * (V**-0.4324)
                EFi_PM2_5 = k * 1.1412 * (V**-0.4324)
            elif 1991 <= car_birth <= 1995:
                if V <= 65.4:
                    EFi_CO = 3.4539 * (V**-0.4266)
                else:
                    EFi_CO = 0.0051 * V + 0.2212
                EFi_HC = 1.6313 * (V**-0.7298)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.5999 * (V**-0.3294)
                EFi_PM2_5 = k * 0.5999 * (V**-0.3294)
            elif 1996 <= car_birth <= 1997:
                if V <= 65.4:
                    EFi_CO = 3.4539 * (V**-0.4266)
                else:
                    EFi_CO = 0.0051 * V + 0.2212
                EFi_HC = 1.1293 * (V**-0.6588)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.6408 * (V**-0.3596)
                EFi_PM2_5 = k * 0.6408 * (V**-0.3596)
            elif 1998 <= car_birth <= 1999:
                EFi_CO = 3.7564 * (V**-0.5175)
                EFi_HC = 1.1293 * (V**-0.6588)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.5168 * (V**-0.3596)
                EFi_PM2_5 = k * 0.5168 * (V**-0.3596)
            elif 2000 <= car_birth <= 2003:
                EFi_CO = 3.7564 * (V**-0.5175)
                EFi_HC = 1.1293 * (V**-0.6588)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.2894 * (V**-0.3596)
                EFi_PM2_5 = k * 0.2894 * (V**-0.3596)
            elif 2004 <= car_birth <= 2007:
                EFi_CO = 3.2797 * (V**-0.8887)
                EFi_HC = 0.1807 * (V**-0.6588)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.2067 * (V**-0.3596)
                EFi_PM2_5 = k * 0.2067 * (V**-0.3596)
            elif 2008 <= car_birth <= 2011:
                if V <= 65.4:
                    EFi_CO = 4.222 * (V**-1.4035)
                else:
                    EFi_CO = 0.011666 * (V**0.09222)
                EFi_HC = 0.829 * (V**-1.0961)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.3111 * (V**-0.5125)
                EFi_PM2_5 = k * 0.3111 * (V**-0.5125)
            elif 2012 <= car_birth:
                if V <= 65.4:
                    EFi_CO = 4.222 * (V**-1.4035)
                else:
                    EFi_CO = 0.011666 * (V**0.09222)
                EFi_HC = 0.829 * (V**-1.0961)
                if V <= 65.4:
                    EFi_NOx = 2.0217 * (V**-0.2645)
                else:
                    EFi_NOx = 0.0271 * (V**0.7596)
                EFi_PM10 = 0.1119 * (V**-0.5125)
                EFi_PM2_5 = k * 0.1119 * (V**-0.5125)
        elif (car_type == '승합') & (car_size == '중형'):
            if car_birth <= 1995:
                EFi_CO = 32.550 * (V**-0.4944)
                EFi_HC = 15.753 * (V**-0.5912)
                if V < 80:
                    EFi_NOx = 40.692 * (V**-0.5590)
                else:
                    EFi_NOx = -0.0023 * (V**2) + 0.5381 * V - 23.590
                EFi_PM10 = 5.4886 * (V**-0.5911)
                EFi_PM2_5 = k * 5.4886 * (V**-0.5911)
            elif 1996 <= car_birth <= 1997:
                EFi_CO = 16.410 * (V**-0.3790)
                EFi_HC = 4.2324 * (V**-0.3926)
                if V < 80:
                    EFi_NOx = 22.804 * (V**-0.4660)
                else:
                    EFi_NOx = -0.0021 * (V**2) + 0.4430 * V - 18.730
                EFi_PM10 = 1.6593 * (V**-0.3935)
                EFi_PM2_5 = k * 1.6593 * (V**-0.3935)
            elif 1998 <= car_birth <= 2000:
                EFi_CO = 16.410 * (V**-0.3790)
                EFi_HC = 4.2324 * (V**-0.3926)
                if V < 80:
                    EFi_NOx = 25.708 * (V**-0.4772)
                else:
                    EFi_NOx = 0.0019 * (V**2) - 0.2628 * V + 12.145
                EFi_PM10 = 1.6593 * (V**-0.3935)
                EFi_PM2_5 = k * 1.6593 * (V**-0.3935)
            elif 2001 <= car_birth <= 2004:
                EFi_CO = 16.378 * (V**-0.5340)
                EFi_HC = 5.8477 * (V**-0.5466)
                if V < 80:
                    EFi_NOx = 25.436 * (V**-0.4656)
                else:
                    EFi_NOx = 0.0008 * (V**2) - 0.0482 * V + 1.8424
                EFi_PM10 = 1.2848 * (V**-0.4715)
                EFi_PM2_5 = k * 1.2848 * (V**-0.4715)
            elif 2005 <= car_birth <= 2007:
                EFi_CO = 15.256 * (V**-0.7448)
                EFi_HC = 2.0502 * (V**-0.6504)
                EFi_NOx = 15.001 * (V**-0.4528)
                EFi_PM10 = 0.2979 * (V**-0.4008)
                EFi_PM2_5 = k * 1.0457 * (V**-0.4527)
            elif 2008 <= car_birth <= 2010:
                EFi_CO = 8.1771 * (V**-0.7725)
                EFi_HC = 1.2991 * (V**-0.6538)
                if V <= 64.7:
                    EFi_NOx = 17.3032 * (V**-0.3660)
                    EFi_PM10 = 0.0539 * (V**-0.5182)
                    EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
                else:
                    EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 2.0000*(10**-4) * V - 7.5600*(10**-3)
                    EFi_PM2_5 = k * 2.0000*(10**-4) * V - 7.5600*(10**-3)
            elif 2011 <= car_birth <= 2014:
                EFi_CO = 4.5201 * (V**-0.7279)
                EFi_HC = 1.6826 * (V**-0.8045)
                if V <= 64.7:
                    EFi_NOx = 17.2485 * (V**-0.4040)
                    EFi_PM10 = 0.0469 * (V**-0.4674)
                    EFi_PM2_5 = k * 0.0469 * (V**-0.4674)
                else:
                    EFi_NOx = 1.1797 * (V**0.2308)
                    EFi_PM10 = 1.6800*(10**-4) * V - 5.1600*(10**-3)
                    EFi_PM2_5 = k * 1.6800*(10**-4) * V - 5.1600*(10**-3)
            elif 2015 <= car_birth:
                EFi_CO = 7.4065 * (V**-0.5995)
                EFi_HC = 2.4562 * (V**-1.3145)
                EFi_NOx = 42.7393 * (V**-1.2949)
                if V <= 64.7:
                    EFi_PM10 = 0.0081 * (V**-0.5182)
                    EFi_PM2_5 = k * 0.0081 * (V**-0.5182)
                else:
                    EFi_PM10 = 3.0000*(10**-5) * V - 0.0011
                    EFi_PM2_5 = k * 3.0000*(10**-5) * V - 0.0011
        elif (car_type == '승합') & (car_size == '대형'):
            if car_birth <= 1995:
                EFi_CO = 28.205 * (V**-0.5337)
                EFi_HC = 6.1146 * (V**-0.4979)
                EFi_NOx = 41.346 * (V**-0.3645)
                EFi_PM10 = 5.2158 * (V**-0.5048)
                EFi_PM2_5 = k * 5.2158 * (V**-0.5048)
            elif 1996 <= car_birth <= 1997:
                EFi_CO = 23.205 * (V**-0.5425)
                EFi_HC = 6.5657 * (V**-0.5431)
                EFi_NOx = 42.1379 * (V**-0.3786)
                EFi_PM10 = 2.4911 * (V**-0.4149)
                EFi_PM2_5 = k * 2.4911 * (V**-0.4149)
            elif 1998 <= car_birth <= 2000:
                EFi_CO = 23.205 * (V**-0.5425)
                EFi_HC = 6.5657 * (V**-0.5431)
                EFi_NOx = 42.1379 * (V**-0.3786)
                EFi_PM10 = 1.4432 * (V**-0.3870)
                EFi_PM2_5 = k * 1.4432 * (V**-0.3870)
            elif car_birth == 2001:
                EFi_CO = 21.348 * (V**-0.5806)
                EFi_HC = 6.6390 * (V**-0.5760)
                EFi_NOx = 36.7191 * (V**-0.3548)
                EFi_PM10 = 0.9375 * (V**-0.3910)
                EFi_PM2_5 = k * 0.9375 * (V**-0.3910)
            elif 2002 <= car_birth <= 2004:
                EFi_CO = 21.348 * (V**-0.5806)
                EFi_HC = 6.6390 * (V**-0.5760)
                EFi_NOx = 36.7191 * (V**-0.3548)
                EFi_PM10 = 1.1507 * (V**-0.4804)
                EFi_PM2_5 = k * 1.1507 * (V**-0.4804)
            elif 2005 <= car_birth <= 2007:
                EFi_CO = 9.6452 * (V**-0.5291)
                EFi_HC = 3.2339 * (V**-0.7436)
                EFi_NOx = 30.5870 * (V**-0.3548)
                if V <= 80:
                    EFi_PM10 = 0.4657 * (V**-0.5634)
                    EFi_PM2_5 = k * 0.4657 * (V**-0.5634)
                else:
                    EFi_PM10 = 0.0014 * (V**0.7970)
                    EFi_PM2_5 = k * 0.0014 * (V**0.7970)
            elif 2008 <= car_birth <= 2010:
                EFi_CO = 6.8493 * (V**-0.6506)
                EFi_HC = 1.7177 * (V**-0.6781)
                EFi_NOx = 40.7564 * (V**-0.4757)
                EFi_PM10 = 0.2418 * (V**-0.4727)
                EFi_PM2_5 = k * 0.2418 * (V**-0.4727)
            elif 2011 <= car_birth <= 2014:
                EFi_CO = 5.4607 * (V**-0.2990)
                EFi_HC = 0.8863 * (V**-0.6933)
                EFi_NOx = 40.3729 * (V**-0.5386)
                EFi_PM10 = 0.2125 * (V**-0.4650)
                EFi_PM2_5 = k * 0.2125 * (V**-0.4650)
            elif 2015 <= car_birth:
                EFi_CO = 11.4415 * (V**-0.8036)
                EFi_HC = 0.6774 * (V**-0.8321)
                EFi_NOx = 112.1229 * (V**-1.6393)
                EFi_PM10 = 0.0363 * (V**-0.4727)
                EFi_PM2_5 = k * 0.0363 * (V**-0.4727)
        elif (car_type == '화물') & (car_size == '소형'):
            if car_birth <= 1990:
                EFi_CO = 4.5854 * (V**-0.3613)
                EFi_HC = 0.4840 * (V**-0.2756)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.8117 * (V**-0.4071)
                EFi_PM2_5 = k * 0.8117 * (V**-0.4071)
            elif 1991 <= car_birth <= 1995:
                EFi_CO = 3.4774 * (V**-0.3483)
                EFi_HC = 0.4844 * (V**-0.3288)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.6188 * (V**-0.4540)
                EFi_PM2_5 = k * 0.6188 * (V**-0.4540)
            elif 1996 <= car_birth <= 1997:
                EFi_CO = 3.3934 * (V**-0.3837)
                EFi_HC = 0.4955 * (V**-0.3393)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.7037 * (V**-0.5357)
                EFi_PM2_5 = k * 0.7037 * (V**-0.5357)
            elif 1998 <= car_birth <= 1999:
                EFi_CO = 4.0896 * (V**-0.6083)
                EFi_HC = 0.6122 * (V**-0.5684)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.6157 * (V**-0.5357)
                EFi_PM2_5 = k * 0.6157 * (V**-0.5357)
            elif 2000 <= car_birth <= 2003:
                EFi_CO = 4.0896 * (V**-0.6083)
                EFi_HC = 0.6122 * (V**-0.5684)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.4838 * (V**-0.5357)
                EFi_PM2_5 = k * 0.4838 * (V**-0.5357)
            elif 2004 <= car_birth <= 2007:
                EFi_CO = 3.2797 * (V**-0.8887)
                EFi_HC = 0.1807 * (V**-0.6588)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.2067 * (V**-0.3596)
                EFi_PM2_5 = k * 0.2067 * (V**-0.3596)
            elif 2008 <= car_birth <= 2011:
                if V <= 65.4:
                    EFi_CO = 4.222 * (V**-1.4035)
                else:
                    EFi_CO = 0.01166 * (V**0.09222)
                EFi_HC = 0.829 * (V**-1.0961)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.3111 * (V**-0.5125)
                EFi_PM2_5 = k * 0.3111 * (V**-0.5125)
            elif 2012 <= car_birth <= 2016:
                if V <= 65.4:
                    EFi_CO = 4.222 * (V**-1.4035)
                    EFi_NOx = 2.0217 * (V**-0.2645)
                else:
                    EFi_CO = 0.01166 * (V**0.09222)
                    EFi_NOx = 0.0271 * (V**0.7596)
                EFi_HC = 0.829 * (V**-1.0961)
                EFi_PM10 = 0.1119 * (V**-0.5125)
                EFi_PM2_5 = k * 0.1119 * (V**-0.5125)
        elif (car_type == '화물') & (car_size == '중형'):
            if car_birth <= 1995:
                EFi_CO = 16.769 * (V**-0.3772)
                EFi_HC = 6.7755 * (V**-0.5003)
                EFi_NOx = 24.915 * (V**-0.3942)
                EFi_PM10 = 3.6772 * (V**-0.5514)
                EFi_PM2_5 = k * 3.6772 * (V**-0.5514)
            elif 1996 <= car_birth <= 2000:
                EFi_CO = 21.057 * (V**-0.4958)
                EFi_HC = 6.7532 * (V**-0.5711)
                EFi_NOx = 25.022 * (V**-0.4240)
                EFi_PM10 = 3.5285 * (V**-0.5962)
                EFi_PM2_5 = k * 3.5285 * (V**-0.5962)
            elif car_birth == 2001:
                EFi_CO = 25.501 * (V**-0.6100)
                EFi_HC = 6.8738 * (V**-0.5913)
                if V <= 64.7:
                    EFi_NOx = 17.3032 * (V**-0.3660)
                else:
                    EFi_NOx = 0.3259 * (V**0.5773)
                EFi_PM10 = 1.4444 * (V**-0.4824)
                EFi_PM2_5 = k * 1.4444 * (V**-0.4824)
            elif 2002 <= car_birth <= 2004:
                EFi_CO = 25.501 * (V**-0.6100)
                EFi_HC = 6.8738 * (V**-0.5913)
                if V <= 64.7:
                    EFi_NOx = 17.3032 * (V**-0.3660)
                else:
                    EFi_NOx = 0.3259 * (V**0.5773)
                EFi_PM10 = 1.0432 * (V**-0.4992)
                EFi_PM2_5 = k * 1.0432 * (V**-0.4992)
            elif 2005 <= car_birth <= 2007:
                EFi_CO = 15.256 * (V**-0.7448)
                EFi_HC = 2.0502 * (V**-0.6504)
                if V <= 64.7:
                    EFi_NOx = 17.3032 * (V**-0.3660)
                else:
                    EFi_NOx = 0.3259 * (V**0.5773)
                EFi_PM10 = 0.2979 * (V**-0.4008)
                EFi_PM2_5 = k * 0.2979 * (V**-0.4008)
            elif 2008 <= car_birth <= 2010:
                EFi_CO = 8.1771 * (V**-0.7725)
                EFi_HC = 1.2991 * (V**-0.6538)
                if V <= 64.7:
                    EFi_NOx = 17.3032 * (V**-0.3660)
                    EFi_PM10 = 0.0539 * (V**-0.5182)
                    EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
                else:
                    EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 0.0002 * V - 0.00756
                    EFi_PM2_5 = k * 0.0002 * V - 0.00756
            elif 2011 <= car_birth <= 2014:
                EFi_CO = 4.5201 * (V**-0.7279)
                EFi_HC = 1.6826 * (V**-0.8045)
                if V <= 64.7:
                    EFi_NOx = 17.2485 * (V**-0.4040)
                    EFi_PM10 = 0.0469 * (V**-0.4674)
                    EFi_PM2_5 = k * 0.0469 * (V**-0.4674)
                else:
                    EFi_NOx = 1.1797 * (V**0.2308)
                    EFi_PM10 = 0.000168 * V - 0.00516
                    EFi_PM2_5 = k * 0.000168 * V - 0.00516
            elif 2015 <= car_birth:
                EFi_CO = 7.4065 * (V**-0.5995)
                EFi_HC = 2.4562 * (V**-1.3145)
                EFi_NOx = 42.7393 * (V**-1.2949)
                if V <= 64.7:
                    EFi_PM10 = 0.0081 * (V**-0.5182)
                    EFi_PM2_5 = k * 0.0081 * (V**-0.5182)
                else:
                    EFi_PM10 = 3.0000 * (10**-5) * V - 0.0011
                    EFi_PM2_5 = k * 3.0000 * (10**-5) * V - 0.0011
        elif (car_type == '화물') & (car_size == '대형'):
            if car_birth <= 1995:
                EFi_CO = 30.402 * (V**-0.4685)
                EFi_HC = 15.75 * (V**-0.582)
                EFi_NOx = 117.49 * (V**-0.365)
                EFi_PM10 = 7.6212 * (V**-0.4183)
                EFi_PM2_5 = k * 7.6212 * (V**-0.4183)
            elif 1996 <= car_birth <= 1997:
                EFi_CO = 18.101 * (V**-0.3454)
                EFi_HC = 10.301 * (V**-0.5856)
                EFi_NOx = 94.319 * (V**-0.4061)
                EFi_PM10 = 6.0264 * (V**-0.4627)
                EFi_PM2_5 = k * 6.0264 * (V**-0.4627)
            elif 1998 <= car_birth <= 2000:
                EFi_CO = 18.101 * (V**-0.3454)
                EFi_HC = 10.301 * (V**-0.5856)
                EFi_NOx = 94.319 * (V**-0.4061)
                EFi_PM10 = 4.873 * (V**-0.4382)
                EFi_PM2_5 = k * 4.873 * (V**-0.4382)
            elif 2001 <= car_birth <= 2004:
                EFi_CO = 28.399 * (V**-0.5999)
                EFi_HC = 10.031 * (V**-0.5828)
                EFi_NOx = 85.301 * (V**-0.4023)
                EFi_PM10 = 3.7541 * (V**-0.4055)
                EFi_PM2_5 = k * 3.7541 * (V**-0.4055)
            elif 2005 <= car_birth <= 2007:
                EFi_CO = 52.136 * (V**-0.8618)
                EFi_HC = 3.7878 * (V**-0.5425)
                EFi_NOx = 107.5 * (V**-0.5679)
                EFi_PM10 = 2.6847 * (V**-0.6112)
                EFi_PM2_5 = k * 2.6847 * (V**-0.6112)
            elif 2008 <= car_birth <= 2010:
                EFi_CO = 6.8493 * (V**-0.6506)
                EFi_HC = 1.7177 * (V**-0.6781)
                EFi_NOx = 40.7564 * (V**-0.4757)
                EFi_PM10 = 0.2418 * (V**-0.4727)
                EFi_PM2_5 = k * 0.2418 * (V**-0.4727)
            elif 2011 <= car_birth <= 2014:
                EFi_CO = 5.4607 * (V**-0.2990)
                EFi_HC = 0.8863 * (V**-0.6933)
                EFi_NOx = 40.3729 * (V**-0.5386)
                EFi_PM10 = 0.2125 * (V**-0.4650)
                EFi_PM2_5 = k * 0.2125 * (V**-0.4650)
            elif 2015 <= car_birth:
                EFi_CO = 4.3762 * (V**-0.4550)
                EFi_HC = 0.3627 * (V**-0.7071)
                EFi_NOx = 18.0405 * (V**-1.0986)
                EFi_PM10 = 0.0363 * (V**-0.4727)
                EFi_PM2_5 = k * 0.0363 * (V**-0.4727)
    elif fuel == 'LPG(액화석유가스)':
        if (car_type == '승용') & (car_size == '경형'):
            if car_birth <= 1996:
                if V <= 45:
                    EFi_CO = 22.498 * (V**-0.6579)
                else:
                    EFi_CO = 0.0006 * (V**2) + 0.0004 * V + 0.8272
                EFi_HC = 12.961 * (V**-0.8364)
                EFi_NOx = 4.0131 * (V**-0.3236)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1997 <= car_birth <= 1999:
                if V <= 45:
                    EFi_CO = 19.887 * (V**-0.8461)
                else:
                    EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                EFi_HC = 2.2714 * (V**-0.7830)
                EFi_NOx = 1.8528 * (V**-0.3889)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2000 <= car_birth <= 2002:
                if V <= 45:
                    EFi_CO = 19.887 * (V**-0.8461)
                else:
                    EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                EFi_HC = 2.2714 * (V**-0.7830)
                EFi_NOx = 5.8289 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2003 <= car_birth <= 2005:
                if V <= 45:
                    EFi_CO = 19.887 * (V**-0.8461)
                else:
                    EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                EFi_HC = 1.1073 * (V**-0.7830)
                EFi_NOx = 5.8289 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2006 <= car_birth <= 2007:
                if V <= 45:
                    EFi_CO = 8.9904 * (V**-0.8461)
                else:
                    EFi_CO = -0.0002 * (V**2) + 0.0457 * V - 1.1403
                EFi_HC = 0.3549 * (V**-0.7830)
                EFi_NOx = 0.7228 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif car_birth == 2008:
                if V <= 79.6:
                    EFi_CO = 0.7693 * (V**-0.7666)
                else:
                    EFi_CO = 5.0000 - 16*(V**7.2766)
    # 수정 중 ...
                if V <= 79.6:
                    EFi_HC = 0.3549 * (V**-0.7830)
                else:
                    EFi_HC = 0.3549 * (V**-0.7830)
                EFi_NOx = 0.7228 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
                
            elif 2009 <= car_birth <= 2011:
                
            elif 2012 <= car_birth <= 2013:
                
            elif car_birth == 2014:
                
            elif 2015 <= car_birth:
                

        elif (car_type == '승용') & (car_size == '소형'):
            if car_birth <= 2004:
                EFi_CO = 5.9672 * (V**-0.9534)
                EFi_HC = 0.6523 * (V**-1.0167)
                EFi_NOx = 24.3491 * (V**-0.7277)
                if V <= 65:
                    EFi_PM10 = 0.3861 * (V**-0.5093)
                    EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                else:
                    EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
            elif car_birth == 2005:
                EFi_CO = 0.7392 * (V**-0.7524)
                EFi_HC = 0.0989 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0839 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0839 * (V**-0.3420)
            elif 2006 <= car_birth <= 2010:
                EFi_CO = 0.5775 * (V**-0.7524)
                EFi_HC = 0.0825 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0420 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0420 * (V**-0.3420)
            elif 2011 <= car_birth <= 2015:
                EFi_CO = 0.5141 * (V**-0.6792)
                EFi_HC = 0.3713 * (V**-0.7513)
                EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
            elif 2016 <= car_birth:
                EFi_CO = 0.4574 * (V**-0.5215)
                EFi_HC = 0.1300 * (V**-0.7265)
                EFi_NOx = 2.7702 * (V**-0.3869)
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
        elif (car_type == '승용') & (car_size == '중형'):
            if car_birth <= 2004:
                EFi_CO = 5.9672 * (V**-0.9534)
                EFi_HC = 0.6523 * (V**-1.0167)
                EFi_NOx = 24.3491 * (V**-0.7277)
                if V <= 65:
                    EFi_PM10 = 0.3861 * (V**-0.5093)
                    EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                else:
                    EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
            elif car_birth == 2005:
                EFi_CO = 0.6930 * (V**-0.7524)
                EFi_HC = 0.1865 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0723 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0723 * (V**-0.3420)
            elif 2006 <= car_birth <= 2010:
                EFi_CO = 0.5414 * (V**-0.7524)
                EFi_HC = 0.0927 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0396 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0396 * (V**-0.3420)
            elif 2011 <= car_birth <= 2015:
                EFi_CO = 0.5141 * (V**-0.6792)
                EFi_HC = 0.3713 * (V**-0.7513)
                EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
            elif 2016 <= car_birth:
                EFi_CO = 0.4574 * (V**-0.5215)
                EFi_HC = 0.1300 * (V**-0.7265)
                EFi_NOx = 2.7702 * (V**-0.3869)
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
        elif (car_type == '승용') & (car_size == '대형'):
            if car_birth <= 2004:
                EFi_CO = 5.9672 * (V**-0.9534)
                EFi_HC = 0.6523 * (V**-1.0167)
                EFi_NOx = 24.3491 * (V**-0.7277)
                if V <= 65:
                    EFi_PM10 = 0.3861 * (V**-0.5093)
                    EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                else:
                    EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
            elif car_birth == 2005:
                EFi_CO = 0.9609 * (V**-0.7524)
                EFi_HC = 0.1865 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0723 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0723 * (V**-0.3420)
            elif 2006 <= car_birth <= 2010:
                EFi_CO = 0.7507 * (V**-0.7524)
                EFi_HC = 0.1554 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0361 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0361 * (V**-0.3420)
            elif 2011 <= car_birth <= 2015:
                EFi_CO = 0.5141 * (V**-0.6792)
                EFi_HC = 0.3713 * (V**-0.7513)
                EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
            elif 2016 <= car_birth:
                EFi_CO = 0.4574 * (V**-0.5215)
                EFi_HC = 0.1300 * (V**-0.7265)
                EFi_NOx = 2.7702 * (V**-0.3869)
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
        elif (car_type == '승합') & (car_size == '소형'):
            if car_birth <= 1990:
                if V <= 65.4:
                    EFi_CO = 3.4539 * (V**-0.4266)
                else:
                    EFi_CO = 0.0051 * V + 0.2212
                EFi_HC = 0.9835 * (V**-0.5096)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 1.1412 * (V**-0.4324)
                EFi_PM2_5 = k * 1.1412 * (V**-0.4324)
            elif 1991 <= car_birth <= 1995:
                if V <= 65.4:
                    EFi_CO = 3.4539 * (V**-0.4266)
                else:
                    EFi_CO = 0.0051 * V + 0.2212
                EFi_HC = 1.6313 * (V**-0.7298)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.5999 * (V**-0.3294)
                EFi_PM2_5 = k * 0.5999 * (V**-0.3294)
            elif 1996 <= car_birth <= 1997:
                if V <= 65.4:
                    EFi_CO = 3.4539 * (V**-0.4266)
                else:
                    EFi_CO = 0.0051 * V + 0.2212
                EFi_HC = 1.1293 * (V**-0.6588)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.6408 * (V**-0.3596)
                EFi_PM2_5 = k * 0.6408 * (V**-0.3596)
            elif 1998 <= car_birth <= 1999:
                EFi_CO = 3.7564 * (V**-0.5175)
                EFi_HC = 1.1293 * (V**-0.6588)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.5168 * (V**-0.3596)
                EFi_PM2_5 = k * 0.5168 * (V**-0.3596)
            elif 2000 <= car_birth <= 2003:
                EFi_CO = 3.7564 * (V**-0.5175)
                EFi_HC = 1.1293 * (V**-0.6588)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.2894 * (V**-0.3596)
                EFi_PM2_5 = k * 0.2894 * (V**-0.3596)
            elif 2004 <= car_birth <= 2007:
                EFi_CO = 3.2797 * (V**-0.8887)
                EFi_HC = 0.1807 * (V**-0.6588)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.2067 * (V**-0.3596)
                EFi_PM2_5 = k * 0.2067 * (V**-0.3596)
            elif 2008 <= car_birth <= 2011:
                if V <= 65.4:
                    EFi_CO = 4.222 * (V**-1.4035)
                else:
                    EFi_CO = 0.011666 * (V**0.09222)
                EFi_HC = 0.829 * (V**-1.0961)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.3111 * (V**-0.5125)
                EFi_PM2_5 = k * 0.3111 * (V**-0.5125)
            elif 2012 <= car_birth:
                if V <= 65.4:
                    EFi_CO = 4.222 * (V**-1.4035)
                else:
                    EFi_CO = 0.011666 * (V**0.09222)
                EFi_HC = 0.829 * (V**-1.0961)
                if V <= 65.4:
                    EFi_NOx = 2.0217 * (V**-0.2645)
                else:
                    EFi_NOx = 0.0271 * (V**0.7596)
                EFi_PM10 = 0.1119 * (V**-0.5125)
                EFi_PM2_5 = k * 0.1119 * (V**-0.5125)
        elif (car_type == '승합') & (car_size == '중형'):
            if car_birth <= 1995:
                EFi_CO = 32.550 * (V**-0.4944)
                EFi_HC = 15.753 * (V**-0.5912)
                if V < 80:
                    EFi_NOx = 40.692 * (V**-0.5590)
                else:
                    EFi_NOx = -0.0023 * (V**2) + 0.5381 * V - 23.590
                EFi_PM10 = 5.4886 * (V**-0.5911)
                EFi_PM2_5 = k * 5.4886 * (V**-0.5911)
            elif 1996 <= car_birth <= 1997:
                EFi_CO = 16.410 * (V**-0.3790)
                EFi_HC = 4.2324 * (V**-0.3926)
                if V < 80:
                    EFi_NOx = 22.804 * (V**-0.4660)
                else:
                    EFi_NOx = -0.0021 * (V**2) + 0.4430 * V - 18.730
                EFi_PM10 = 1.6593 * (V**-0.3935)
                EFi_PM2_5 = k * 1.6593 * (V**-0.3935)
            elif 1998 <= car_birth <= 2000:
                EFi_CO = 16.410 * (V**-0.3790)
                EFi_HC = 4.2324 * (V**-0.3926)
                if V < 80:
                    EFi_NOx = 25.708 * (V**-0.4772)
                else:
                    EFi_NOx = 0.0019 * (V**2) - 0.2628 * V + 12.145
                EFi_PM10 = 1.6593 * (V**-0.3935)
                EFi_PM2_5 = k * 1.6593 * (V**-0.3935)
            elif 2001 <= car_birth <= 2004:
                EFi_CO = 16.378 * (V**-0.5340)
                EFi_HC = 5.8477 * (V**-0.5466)
                if V < 80:
                    EFi_NOx = 25.436 * (V**-0.4656)
                else:
                    EFi_NOx = 0.0008 * (V**2) - 0.0482 * V + 1.8424
                EFi_PM10 = 1.2848 * (V**-0.4715)
                EFi_PM2_5 = k * 1.2848 * (V**-0.4715)
            elif 2005 <= car_birth <= 2007:
                EFi_CO = 15.256 * (V**-0.7448)
                EFi_HC = 2.0502 * (V**-0.6504)
                EFi_NOx = 15.001 * (V**-0.4528)
                EFi_PM10 = 0.2979 * (V**-0.4008)
                EFi_PM2_5 = k * 1.0457 * (V**-0.4527)
            elif 2008 <= car_birth <= 2010:
                EFi_CO = 8.1771 * (V**-0.7725)
                EFi_HC = 1.2991 * (V**-0.6538)
                if V <= 64.7:
                    EFi_NOx = 17.3032 * (V**-0.3660)
                    EFi_PM10 = 0.0539 * (V**-0.5182)
                    EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
                else:
                    EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 2.0000*(10**-4) * V - 7.5600*(10**-3)
                    EFi_PM2_5 = k * 2.0000*(10**-4) * V - 7.5600*(10**-3)
            elif 2011 <= car_birth <= 2014:
                EFi_CO = 4.5201 * (V**-0.7279)
                EFi_HC = 1.6826 * (V**-0.8045)
                if V <= 64.7:
                    EFi_NOx = 17.2485 * (V**-0.4040)
                    EFi_PM10 = 0.0469 * (V**-0.4674)
                    EFi_PM2_5 = k * 0.0469 * (V**-0.4674)
                else:
                    EFi_NOx = 1.1797 * (V**0.2308)
                    EFi_PM10 = 1.6800*(10**-4) * V - 5.1600*(10**-3)
                    EFi_PM2_5 = k * 1.6800*(10**-4) * V - 5.1600*(10**-3)
            elif 2015 <= car_birth:
                EFi_CO = 7.4065 * (V**-0.5995)
                EFi_HC = 2.4562 * (V**-1.3145)
                EFi_NOx = 42.7393 * (V**-1.2949)
                if V <= 64.7:
                    EFi_PM10 = 0.0081 * (V**-0.5182)
                    EFi_PM2_5 = k * 0.0081 * (V**-0.5182)
                else:
                    EFi_PM10 = 3.0000*(10**-5) * V - 0.0011
                    EFi_PM2_5 = k * 3.0000*(10**-5) * V - 0.0011
        elif (car_type == '승합') & (car_size == '대형'):
            if car_birth <= 1995:
                EFi_CO = 28.205 * (V**-0.5337)
                EFi_HC = 6.1146 * (V**-0.4979)
                EFi_NOx = 41.346 * (V**-0.3645)
                EFi_PM10 = 5.2158 * (V**-0.5048)
                EFi_PM2_5 = k * 5.2158 * (V**-0.5048)
            elif 1996 <= car_birth <= 1997:
                EFi_CO = 23.205 * (V**-0.5425)
                EFi_HC = 6.5657 * (V**-0.5431)
                EFi_NOx = 42.1379 * (V**-0.3786)
                EFi_PM10 = 2.4911 * (V**-0.4149)
                EFi_PM2_5 = k * 2.4911 * (V**-0.4149)
            elif 1998 <= car_birth <= 2000:
                EFi_CO = 23.205 * (V**-0.5425)
                EFi_HC = 6.5657 * (V**-0.5431)
                EFi_NOx = 42.1379 * (V**-0.3786)
                EFi_PM10 = 1.4432 * (V**-0.3870)
                EFi_PM2_5 = k * 1.4432 * (V**-0.3870)
            elif car_birth == 2001:
                EFi_CO = 21.348 * (V**-0.5806)
                EFi_HC = 6.6390 * (V**-0.5760)
                EFi_NOx = 36.7191 * (V**-0.3548)
                EFi_PM10 = 0.9375 * (V**-0.3910)
                EFi_PM2_5 = k * 0.9375 * (V**-0.3910)
            elif 2002 <= car_birth <= 2004:
                EFi_CO = 21.348 * (V**-0.5806)
                EFi_HC = 6.6390 * (V**-0.5760)
                EFi_NOx = 36.7191 * (V**-0.3548)
                EFi_PM10 = 1.1507 * (V**-0.4804)
                EFi_PM2_5 = k * 1.1507 * (V**-0.4804)
            elif 2005 <= car_birth <= 2007:
                EFi_CO = 9.6452 * (V**-0.5291)
                EFi_HC = 3.2339 * (V**-0.7436)
                EFi_NOx = 30.5870 * (V**-0.3548)
                if V <= 80:
                    EFi_PM10 = 0.4657 * (V**-0.5634)
                    EFi_PM2_5 = k * 0.4657 * (V**-0.5634)
                else:
                    EFi_PM10 = 0.0014 * (V**0.7970)
                    EFi_PM2_5 = k * 0.0014 * (V**0.7970)
            elif 2008 <= car_birth <= 2010:
                EFi_CO = 6.8493 * (V**-0.6506)
                EFi_HC = 1.7177 * (V**-0.6781)
                EFi_NOx = 40.7564 * (V**-0.4757)
                EFi_PM10 = 0.2418 * (V**-0.4727)
                EFi_PM2_5 = k * 0.2418 * (V**-0.4727)
            elif 2011 <= car_birth <= 2014:
                EFi_CO = 5.4607 * (V**-0.2990)
                EFi_HC = 0.8863 * (V**-0.6933)
                EFi_NOx = 40.3729 * (V**-0.5386)
                EFi_PM10 = 0.2125 * (V**-0.4650)
                EFi_PM2_5 = k * 0.2125 * (V**-0.4650)
            elif 2015 <= car_birth:
                EFi_CO = 11.4415 * (V**-0.8036)
                EFi_HC = 0.6774 * (V**-0.8321)
                EFi_NOx = 112.1229 * (V**-1.6393)
                EFi_PM10 = 0.0363 * (V**-0.4727)
                EFi_PM2_5 = k * 0.0363 * (V**-0.4727)
        elif (car_type == '화물') & (car_size == '소형'):
            if car_birth <= 1990:
                EFi_CO = 4.5854 * (V**-0.3613)
                EFi_HC = 0.4840 * (V**-0.2756)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.8117 * (V**-0.4071)
                EFi_PM2_5 = k * 0.8117 * (V**-0.4071)
            elif 1991 <= car_birth <= 1995:
                EFi_CO = 3.4774 * (V**-0.3483)
                EFi_HC = 0.4844 * (V**-0.3288)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.6188 * (V**-0.4540)
                EFi_PM2_5 = k * 0.6188 * (V**-0.4540)
            elif 1996 <= car_birth <= 1997:
                EFi_CO = 3.3934 * (V**-0.3837)
                EFi_HC = 0.4955 * (V**-0.3393)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.7037 * (V**-0.5357)
                EFi_PM2_5 = k * 0.7037 * (V**-0.5357)
            elif 1998 <= car_birth <= 1999:
                EFi_CO = 4.0896 * (V**-0.6083)
                EFi_HC = 0.6122 * (V**-0.5684)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.6157 * (V**-0.5357)
                EFi_PM2_5 = k * 0.6157 * (V**-0.5357)
            elif 2000 <= car_birth <= 2003:
                EFi_CO = 4.0896 * (V**-0.6083)
                EFi_HC = 0.6122 * (V**-0.5684)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.4838 * (V**-0.5357)
                EFi_PM2_5 = k * 0.4838 * (V**-0.5357)
            elif 2004 <= car_birth <= 2007:
                EFi_CO = 3.2797 * (V**-0.8887)
                EFi_HC = 0.1807 * (V**-0.6588)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.2067 * (V**-0.3596)
                EFi_PM2_5 = k * 0.2067 * (V**-0.3596)
            elif 2008 <= car_birth <= 2011:
                if V <= 65.4:
                    EFi_CO = 4.222 * (V**-1.4035)
                else:
                    EFi_CO = 0.01166 * (V**0.09222)
                EFi_HC = 0.829 * (V**-1.0961)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.3111 * (V**-0.5125)
                EFi_PM2_5 = k * 0.3111 * (V**-0.5125)
            elif 2012 <= car_birth <= 2016:
                if V <= 65.4:
                    EFi_CO = 4.222 * (V**-1.4035)
                    EFi_NOx = 2.0217 * (V**-0.2645)
                else:
                    EFi_CO = 0.01166 * (V**0.09222)
                    EFi_NOx = 0.0271 * (V**0.7596)
                EFi_HC = 0.829 * (V**-1.0961)
                EFi_PM10 = 0.1119 * (V**-0.5125)
                EFi_PM2_5 = k * 0.1119 * (V**-0.5125)
        elif (car_type == '화물') & (car_size == '중형'):
            if car_birth <= 1995:
                EFi_CO = 16.769 * (V**-0.3772)
                EFi_HC = 6.7755 * (V**-0.5003)
                EFi_NOx = 24.915 * (V**-0.3942)
                EFi_PM10 = 3.6772 * (V**-0.5514)
                EFi_PM2_5 = k * 3.6772 * (V**-0.5514)
            elif 1996 <= car_birth <= 2000:
                EFi_CO = 21.057 * (V**-0.4958)
                EFi_HC = 6.7532 * (V**-0.5711)
                EFi_NOx = 25.022 * (V**-0.4240)
                EFi_PM10 = 3.5285 * (V**-0.5962)
                EFi_PM2_5 = k * 3.5285 * (V**-0.5962)
            elif car_birth == 2001:
                EFi_CO = 25.501 * (V**-0.6100)
                EFi_HC = 6.8738 * (V**-0.5913)
                if V <= 64.7:
                    EFi_NOx = 17.3032 * (V**-0.3660)
                else:
                    EFi_NOx = 0.3259 * (V**0.5773)
                EFi_PM10 = 1.4444 * (V**-0.4824)
                EFi_PM2_5 = k * 1.4444 * (V**-0.4824)
            elif 2002 <= car_birth <= 2004:
                EFi_CO = 25.501 * (V**-0.6100)
                EFi_HC = 6.8738 * (V**-0.5913)
                if V <= 64.7:
                    EFi_NOx = 17.3032 * (V**-0.3660)
                else:
                    EFi_NOx = 0.3259 * (V**0.5773)
                EFi_PM10 = 1.0432 * (V**-0.4992)
                EFi_PM2_5 = k * 1.0432 * (V**-0.4992)
            elif 2005 <= car_birth <= 2007:
                EFi_CO = 15.256 * (V**-0.7448)
                EFi_HC = 2.0502 * (V**-0.6504)
                if V <= 64.7:
                    EFi_NOx = 17.3032 * (V**-0.3660)
                else:
                    EFi_NOx = 0.3259 * (V**0.5773)
                EFi_PM10 = 0.2979 * (V**-0.4008)
                EFi_PM2_5 = k * 0.2979 * (V**-0.4008)
            elif 2008 <= car_birth <= 2010:
                EFi_CO = 8.1771 * (V**-0.7725)
                EFi_HC = 1.2991 * (V**-0.6538)
                if V <= 64.7:
                    EFi_NOx = 17.3032 * (V**-0.3660)
                    EFi_PM10 = 0.0539 * (V**-0.5182)
                    EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
                else:
                    EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 0.0002 * V - 0.00756
                    EFi_PM2_5 = k * 0.0002 * V - 0.00756
            elif 2011 <= car_birth <= 2014:
                EFi_CO = 4.5201 * (V**-0.7279)
                EFi_HC = 1.6826 * (V**-0.8045)
                if V <= 64.7:
                    EFi_NOx = 17.2485 * (V**-0.4040)
                    EFi_PM10 = 0.0469 * (V**-0.4674)
                    EFi_PM2_5 = k * 0.0469 * (V**-0.4674)
                else:
                    EFi_NOx = 1.1797 * (V**0.2308)
                    EFi_PM10 = 0.000168 * V - 0.00516
                    EFi_PM2_5 = k * 0.000168 * V - 0.00516
            elif 2015 <= car_birth:
                EFi_CO = 7.4065 * (V**-0.5995)
                EFi_HC = 2.4562 * (V**-1.3145)
                EFi_NOx = 42.7393 * (V**-1.2949)
                if V <= 64.7:
                    EFi_PM10 = 0.0081 * (V**-0.5182)
                    EFi_PM2_5 = k * 0.0081 * (V**-0.5182)
                else:
                    EFi_PM10 = 3.0000 * (10**-5) * V - 0.0011
                    EFi_PM2_5 = k * 3.0000 * (10**-5) * V - 0.0011
        elif (car_type == '화물') & (car_size == '대형'):
            if car_birth <= 1995:
                EFi_CO = 30.402 * (V**-0.4685)
                EFi_HC = 15.75 * (V**-0.582)
                EFi_NOx = 117.49 * (V**-0.365)
                EFi_PM10 = 7.6212 * (V**-0.4183)
                EFi_PM2_5 = k * 7.6212 * (V**-0.4183)
            elif 1996 <= car_birth <= 1997:
                EFi_CO = 18.101 * (V**-0.3454)
                EFi_HC = 10.301 * (V**-0.5856)
                EFi_NOx = 94.319 * (V**-0.4061)
                EFi_PM10 = 6.0264 * (V**-0.4627)
                EFi_PM2_5 = k * 6.0264 * (V**-0.4627)
            elif 1998 <= car_birth <= 2000:
                EFi_CO = 18.101 * (V**-0.3454)
                EFi_HC = 10.301 * (V**-0.5856)
                EFi_NOx = 94.319 * (V**-0.4061)
                EFi_PM10 = 4.873 * (V**-0.4382)
                EFi_PM2_5 = k * 4.873 * (V**-0.4382)
            elif 2001 <= car_birth <= 2004:
                EFi_CO = 28.399 * (V**-0.5999)
                EFi_HC = 10.031 * (V**-0.5828)
                EFi_NOx = 85.301 * (V**-0.4023)
                EFi_PM10 = 3.7541 * (V**-0.4055)
                EFi_PM2_5 = k * 3.7541 * (V**-0.4055)
            elif 2005 <= car_birth <= 2007:
                EFi_CO = 52.136 * (V**-0.8618)
                EFi_HC = 3.7878 * (V**-0.5425)
                EFi_NOx = 107.5 * (V**-0.5679)
                EFi_PM10 = 2.6847 * (V**-0.6112)
                EFi_PM2_5 = k * 2.6847 * (V**-0.6112)
            elif 2008 <= car_birth <= 2010:
                EFi_CO = 6.8493 * (V**-0.6506)
                EFi_HC = 1.7177 * (V**-0.6781)
                EFi_NOx = 40.7564 * (V**-0.4757)
                EFi_PM10 = 0.2418 * (V**-0.4727)
                EFi_PM2_5 = k * 0.2418 * (V**-0.4727)
            elif 2011 <= car_birth <= 2014:
                EFi_CO = 5.4607 * (V**-0.2990)
                EFi_HC = 0.8863 * (V**-0.6933)
                EFi_NOx = 40.3729 * (V**-0.5386)
                EFi_PM10 = 0.2125 * (V**-0.4650)
                EFi_PM2_5 = k * 0.2125 * (V**-0.4650)
            elif 2015 <= car_birth:
                EFi_CO = 4.3762 * (V**-0.4550)
                EFi_HC = 0.3627 * (V**-0.7071)
                EFi_NOx = 18.0405 * (V**-1.0986)
                EFi_PM10 = 0.0363 * (V**-0.4727)
                EFi_PM2_5 = k * 0.0363 * (V**-0.4727)

# 코드 마지막